<a href="https://colab.research.google.com/github/Filarh/Hello-World/blob/master/DeFooocusfinal_fixed_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check if 'Args' is in the local scope
if 'Args' not in locals():
    # Install dependencies if necessary
    !apt install -y aria2
    !pip install pygit2==1.12.2
    %cd /content
    !git clone https://github.com/ehristoforu/DeFooocus.git
    %cd /content/DeFooocus
    Args = ""

# Import necessary libraries
import os
import json
from google.colab import drive

# Fix: Create the 'cached_download' folder in the 'huggingface_hub' path if it doesn't exist
import sys
huggingface_hub_path = '/usr/local/lib/python3.10/dist-packages/huggingface_hub'
cached_download_path = os.path.join(huggingface_hub_path, "cached_download")
if not os.path.exists(cached_download_path):
    os.makedirs(cached_download_path)

# Function to print in color (Colab compatible)
def print_colored(text, color):
    color_codes = {
        "red": "\033[91m",
        "green": "\033[92m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
        "reset": "\033[0m"
    }
    print(f"{color_codes.get(color, color_codes['reset'])}{text}{color_codes['reset']}")

# Function to modify JSON with provided parameters
def modificar_json(nombre_archivo: str, link_archivo: str, aspect_ratio: str, formato: str):
    carpeta = '/content/DeFooocus/presets'
    archivos = os.listdir(carpeta)

    json_encontrado = None
    if 'default.json' in archivos:
        json_encontrado = 'default.json'
        print_colored("Using default.json as a reference", "green")
    else:
        json_encontrado = next((archivo for archivo in archivos if archivo.endswith('.json')), None)
        print_colored("No default.json found, using a random JSON file.", "yellow")

    if json_encontrado:
        with open(os.path.join(carpeta, json_encontrado)) as file:
            data = json.load(file)

        archivo_modelo = f"{nombre_archivo}.{formato}"
        data['default_model'] = archivo_modelo
        data['default_aspect_ratio'] = aspect_ratio
        data['default_refiner'] = ""

        data['checkpoint_downloads'] = {archivo_modelo: link_archivo}

        global Args
        Args = f"--preset {nombre_archivo}"

        nuevo_nombre = f"{nombre_archivo}.json"
        with open(os.path.join(carpeta, nuevo_nombre), 'w') as file:
            json.dump(data, file, separators=(',', ':'))

        return f"Created {nuevo_nombre} with the provided parameters and structure!"
    else:
        return "No JSON files found in the specified folder."

# Function to download the model using aria2 if it does not exist
def download_model_aria2(url, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE):
    try:
        model_dir = "/content/DeFooocus/models/checkpoints"
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        archivo_modelo = f"{nombre_archivo}.{formato}"
        destino = os.path.join(model_dir, archivo_modelo)

        if os.path.exists(destino):
            print_colored(f"File {archivo_modelo} already exists. No download required.", "cyan")
            return destino

        if "huggingface" in url and tokenHUGGINGFACE:
            url += f"&token={tokenHUGGINGFACE}"
        elif "civitai" in url and tokenCIVITAI:
            url += f"&token={tokenCIVITAI}"

        download_command = f'aria2c -x 16 -s 16 -k 1M "{url}" -d "{model_dir}" -o "{archivo_modelo}"'

        print_colored(f"Downloading from URL: {download_command}", "blue")
        os.system(download_command)

        if os.path.exists(destino):
            print_colored(f"Downloaded file at: {destino}", "green")
            return destino
        else:
            print_colored("Failed to download file.", "red")
            return None

    except Exception as e:
        print_colored(f"Download error: {e}", "red")
        return None

# Activating Google Drive if necessary
activar_drive = True
if activar_drive:
    drive.mount('/content/drive')
    nuevo_dir_lora = '/content/drive/MyDrive/MyLoras'
    nuevo_dir_salida = '/content/drive/MyDrive/Fooocus_output'

    launch_py_path = '/content/DeFooocus/launch.py'

    if os.path.exists(launch_py_path):
        with open(launch_py_path, 'r') as file:
            launch_py_content = file.read()

        codigo_a_insertar = f'''
import os
import json

nuevo_dir_lora = '{nuevo_dir_lora}'
nuevo_dir_salida = '{nuevo_dir_salida}'

config_path = os.path.abspath("./config.txt")
config_dict = {{}}

if os.path.exists(config_path):
    with open(config_path, "r", encoding="utf-8") as json_file:
        config_dict = json.load(json_file)

config_dict['path_loras'] = os.path.abspath(nuevo_dir_lora)
config_dict['path_outputs'] = os.path.abspath(nuevo_dir_salida)

with open(config_path, "w", encoding="utf-8") as json_file:
    json.dump(config_dict, json_file, indent=4)
'''

        if codigo_a_insertar not in launch_py_content:
            with open(launch_py_path, 'w') as file:
                file.write(codigo_a_insertar + launch_py_content)
        else:
            print_colored("Code already inserted in launch.py", "cyan")
    else:
        print_colored("launch.py file not found.", "red")

# JSON model modification
modificar_modelo = True
nombre_archivo = "ZavyChromaXL"
link_archivo = "https://civitai.com/api/download/models/916744?type=Model&format=SafeTensor&size=full&fp=fp16"
aspect_ratio = "1024*1024"
formato = "safetensors"

if modificar_modelo:
    resultado = modificar_json(nombre_archivo, link_archivo, aspect_ratio, formato)
    print(resultado)

model_url = link_archivo
tokenCIVITAI = ""
tokenHUGGINGFACE = ""

model_sd = download_model_aria2(model_url, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE)
if model_sd:
    print_colored(f"Model downloaded: {model_sd}", "green")
else:
    print_colored("Model download failed.", "red")

print_colored("[DeFooocus] Preparing...", "magenta")

theme = "dark"
preset = "default"
advenced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16"

if preset != "default":
    args = f"{advenced_args} --theme {theme} --preset {preset}"
else:
    args = f"{advenced_args} --theme {theme}"

args = f"{args} {Args}"

print_colored("[DeFooocus] Starting...", "magenta")
!python entry_with_update.py {args}


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 2s (942 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 124561 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu